In [1]:
%run data_setup

In [2]:
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

import numpyro
from numpyro.infer import SVI, Trace_ELBO, MCMC, NUTS, Predictive
import numpyro.distributions as dist
from numpyro.infer.reparam import NeuTraReparam, LocScaleReparam
from numpyro.infer.autoguide import AutoDiagonalNormal
from numpyro.diagnostics import split_gelman_rubin
from numpyro.handlers import reparam
from numpyro.optim import Adam
import jax
import jax.numpy as jnp
from jax import random, device_get

numpyro.util.set_host_device_count(5)
device_num = jax.device_count('cpu')
numpyro.enable_x64()
prng_seed = random.PRNGKey(33425453)

# Traditional Models

## Ordinary least squares for simple models

### Gravity
$$
\hat M_{i,j,t} = \kappa \frac{ P_{i,t} P_{j,t} } { D_{i,j}}
$$

In [4]:
path_count = 5

In [5]:
cpc_gravity , cpcd_gravity, mae_gravity , r_squared_gravity = np.empty(path_count), np.empty(path_count), np.empty(path_count), np.empty(path_count)
for path in df.path_ind.unique()[:path_count]:

    df_train_filtered = df_train.query("path_ind == @path & M_ij != 0")
    X_train = df_train_filtered[['P_i','P_j','D_ij']].astype('float')
    y_train = df_train_filtered.M_ij
    gravity_ols = sm.OLS(np.log(y_train), sm.add_constant(np.log(X_train))).fit()

    df_test_filtered = df_test.query("path_ind == @path & M_ij != 0")
    X_test = df_test_filtered[['P_i','P_j','D_ij']].astype('float')
    y_test = df_test_filtered.M_ij

    pred = np.exp( gravity_ols.predict( sm.add_constant(np.log(X_test)) ) )

    cpc_gravity[path] = cpc(y_test, pred) 
    cpcd_gravity[path] = cpcd(y_test, pred, df_test_filtered.D_ij) 
    mae_gravity[path] = mae(y_test, pred) 
    r_squared_gravity[path] = r_squared(y_test, pred) 

print("MAE:", mae_gravity.mean()            , "+/-", norm.ppf(.975) * mae_gravity.std(ddof=1) / np.sqrt(path_count)      )
print("R-squared:", r_squared_gravity.mean(), "+/-", norm.ppf(.975) * r_squared_gravity.std(ddof=1) / np.sqrt(path_count)      )
print("CPC:", cpc_gravity.mean()             , "+/-", norm.ppf(.975) * cpc_gravity.std(ddof=1) / np.sqrt(path_count)      )
print("CPCD:", cpcd_gravity.mean()           , "+/-", norm.ppf(.975) * cpcd_gravity.std(ddof=1) / np.sqrt(path_count)      )
# print("BIC:", gravity_ols.bic)

MAE: 1894.3986174749848 +/- 7.201791653105702
R-squared: 0.4019724092629852 +/- 0.0009483982499523122
CPC: 0.6337717765697434 +/- 0.0012180169622381947
CPCD: 0.7563012823319919 +/- 0.0009056481365264334


In [6]:
gravity_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   M_ij   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.509
Method:                 Least Squares   F-statistic:                     9809.
Date:                Wed, 08 Nov 2023   Prob (F-statistic):               0.00
Time:                        17:07:22   Log-Likelihood:                -46128.
No. Observations:               28432   AIC:                         9.226e+04
Df Residuals:                   28428   BIC:                         9.230e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.3496      0.176    -70.269      0.000     -12.694     -12.005
P_i            0.7673      0.007    107.588      0.000       0.753       0.781
P_j            0.7883      0.007    111.096      0.000       0.774       0.802
D_ij          -0.6102      0.009    -66.760      0.000      -0.628      -0.592
==============================================================================
Omnibus:                     2827.209   Durbin-Watson:                   1.697
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5291.449
Skew:                          -0.674   Prob(JB):                         0.00
Kurtosis:                       4.627   Cond. No.                         549.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Radiation
$$
\hat M_{i,j,t} = \hat M_{i,t} \frac { P_{i,t} P_{j,t}  } {  (P_{i,t} + S_{i,j,t}) (P_{i,t} + P_{j,t} + S_{i,j,t})  }
$$

In [7]:
cpc_r , cpcd_r, mae_r , r_squared_r = np.empty(path_count), np.empty(path_count), np.empty(path_count), np.empty(path_count)
for path in df.path_ind.unique()[:path_count]:

    df_train_filtered = df_train.query("path_ind == @path & M_ij != 0")
    X_train = df_train_filtered[['P_i','P_j','SP_ij']].astype('float')
    X_train['P_i + SP_ij'] = X_train.P_i + X_train.SP_ij
    X_train['P_i + P_j + SP_ij'] = X_train.P_i + X_train.P_j + X_train.SP_ij
    X_train.drop('SP_ij',axis=1,inplace=True)
    y_train = df_train_filtered.M_ij

    rad_ols = sm.OLS(np.log(y_train), sm.add_constant(np.log(X_train))).fit()

    df_test_filtered = df_test.query("path_ind == @path & M_ij != 0")
    X_test = df_test_filtered[['P_i','P_j','SP_ij']].astype('float')
    X_test['P_i + SP_ij'] = X_test.P_i + X_test.SP_ij
    X_test['P_i + P_j + SP_ij'] = X_test.P_i + X_test.P_j + X_test.SP_ij
    X_test.drop('SP_ij',axis=1,inplace=True)    
    y_test = df_test_filtered.M_ij

    pred = np.exp( rad_ols.predict( sm.add_constant(np.log(X_test)) ))

    cpc_r[path] =  cpc(y_test, pred)
    cpcd_r[path] = cpcd(y_test, pred, df_test_filtered.D_ij)
    mae_r[path] = mae(y_test, pred)
    r_squared_r[path] = r_squared(y_test, pred)

print("MAE:", mae_r.mean()            , "+/-", norm.ppf(.975) * mae_r.std(ddof=1) / np.sqrt(path_count)      )
print("R-squared:", r_squared_r.mean(), "+/-", norm.ppf(.975) * r_squared_r.std(ddof=1) / np.sqrt(path_count)      )
print("CPC:", cpc_r.mean()             , "+/-", norm.ppf(.975) * cpc_r.std(ddof=1) / np.sqrt(path_count)      )
print("CPCD:", cpcd_r.mean()           , "+/-", norm.ppf(.975) * cpcd_r.std(ddof=1) / np.sqrt(path_count)      )
# print("BIC:", rad_ols.bic)

MAE: 1657.7314500935186 +/- 5.830838789400446
R-squared: 0.5274364369669536 +/- 0.002694356279335885
CPC: 0.6951472820854869 +/- 0.0010415672821489576
CPCD: 0.816930285281288 +/- 0.0016372229054317463


In [8]:
rad_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   M_ij   R-squared:                       0.592
Model:                            OLS   Adj. R-squared:                  0.592
Method:                 Least Squares   F-statistic:                 1.029e+04
Date:                Wed, 08 Nov 2023   Prob (F-statistic):               0.00
Time:                        17:08:32   Log-Likelihood:                -43499.
No. Observations:               28432   AIC:                         8.701e+04
Df Residuals:                   28427   BIC:                         8.705e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -5.5948      0.200    -27.980      0.000      -5.987      -5.203
P_i                   0.8808      0.007    134.670      0.000       0.868       0.894
P_j                   0.8029      0.007    115.460      0.000       0.789       0.817
P_i + SP_ij          -0.5171      0.039    -13.101      0.000      -0.594      -0.440
P_i + P_j + SP_ij    -0.1918      0.045     -4.244      0.000      -0.280      -0.103
==============================================================================
Omnibus:                     3926.421   Durbin-Watson:                   1.800
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8836.344
Skew:                          -0.821   Prob(JB):                         0.00
Kurtosis:                       5.183   Cond. No.                     1.03e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.03e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
from statsmodels.compat import lzip

# Conduct the Breusch-Pagan test
names = ['Lagrange multiplier statistic', 'p-value',
         'f-value', 'f p-value']

test_result = sm.stats.het_breuschpagan(rad_ols.resid, rad_ols.model.exog)
 
lzip(names, test_result)

[('Lagrange multiplier statistic', 591.2844606665697),
 ('p-value', 1.192459743295018e-126),
 ('f-value', 150.7091169095696),
 ('f p-value', 6.931790228605662e-128)]

## Bayesian Inference for Generalized Models

### Generalized Gravity

In [13]:
def ggm(p_i, d_ij, y, scaled_factors_origin, scaled_factors_destination):

    λ = numpyro.sample("λ", dist.HalfNormal(1))
    w = numpyro.sample("w", dist.Laplace(0, λ * jnp.ones(scaled_factors_origin.shape[1])))
    
    U_i = jnp.dot(w , scaled_factors_origin.T)
    U_j = jnp.dot(w , scaled_factors_destination.T)

    Β = jnp.array([1,1,1,1,-1]) * numpyro.sample("Β", dist.HalfNormal(1*jnp.ones(5))) 
    σ = numpyro.sample("σ", dist.HalfNormal(1))

    M_ij = Β[0] + jnp.dot(Β[1:], jnp.array([jnp.log(1+p_i), jnp.log(1+U_i), jnp.log(1+U_j), jnp.log(1 + d_ij )]))

    numpyro.sample("flow", dist.Normal(M_ij, σ), obs=y)

In [3]:
def ggm(p_i, d_ij, y, scaled_factors_origin, scaled_factors_destination):

    λ = numpyro.sample("λ", dist.HalfNormal(1))
    w = numpyro.sample("w", dist.Laplace(0, λ * jnp.ones(scaled_factors_origin.shape[1])))
    
    U_i = jnp.dot(w , scaled_factors_origin.T)
    U_j = jnp.dot(w , scaled_factors_destination.T)

    # factors = jnp.array([jnp.log(1+p_i), jnp.log(1+U_i), jnp.log(1+U_j), jnp.log(1 + d_ij )])

    # Β_param = np.linalg.inv(factors.T @ factors) @ factors.T @ y
    # α_param = y.mean() - Β_param @ factors.mean(axis=0)

    Β = numpyro.sample("Β", dist.Normal(np.zeros(5),1)) 
    σ = numpyro.sample("σ", dist.HalfNormal(1))

    M_ij = Β[0] + jnp.dot(Β[1:], jnp.array([jnp.log(1+p_i), jnp.log(1+U_i), jnp.log(1+U_j), jnp.log(1 + d_ij )]))

    numpyro.sample("flow", dist.Normal(M_ij, σ), obs=y)

In [4]:
cpc_gg , cpcd_gg, mae_gg , r_squared_gg = np.empty(path_count), np.empty(path_count), np.empty(path_count), np.empty(path_count)
# weights_gg, coeff_gg = np.empty((path_count, len(features))), np.empty((path_count, 4))
posteriors_gg = []
for path in df.path_ind.unique()[:path_count]:

    X_train = df_train.query("path_ind == @path").loc[:,'P_i':].astype('float')
    y_train = np.log(1 + df_train.query("path_ind == @path").M_ij.values)

    tmp = np.log(1 + X_train)
    scaled_factors = 1 / (1 + np.exp(- (tmp - tmp.mean(axis=0)) / tmp.std(ddof=1, axis=0) ))

    features = {'population': ['P_i','P_j'],
            'housing': ['H_i','H_j'],
            'income': ['I_i','I_j'],
            'affordability': ['AF_i','AF_j'],
            'area': ['A_i','A_j'],
            'density': ['rho_i','rho_j'],
            'climate risk': ['ADC_i','ADC_j']
            }

    features_origin = list(np.array(list(features.values()))[:,0])
    features_destination = list(np.array(list(features.values()))[:,1])

    scaled_factors_origin = scaled_factors[features_origin].values
    scaled_factors_destination = scaled_factors[features_destination].values

    # config = {"w":LocScaleReparam(), "Β":LocScaleReparam()}
    # reparam_ggm = reparam(ggm, config)   
    guide = AutoDiagonalNormal(ggm)
    svi = SVI(ggm, guide, Adam(1e-4), Trace_ELBO())
    svi_result = svi.run(prng_seed, 25000, X_train.P_i.values, X_train.D_ij.values, y_train, scaled_factors_origin, scaled_factors_destination, progress_bar=True)
    neutra = NeuTraReparam(guide, svi_result.params)

    reparam_ggm = neutra.reparam(ggm)
    # config = {"w":LocScaleReparam(), "Β":LocScaleReparam()}
    # reparam_ggm = reparam(reparam_ggm, config)
    nuts_kernel = NUTS(reparam_ggm, init_strategy=numpyro.infer.init_to_sample, target_accept_prob=.99, max_tree_depth=10)
    num_warmup = 200
    num_samples = 1000
    num_chains = 5
    thinning = 1

    mcmc = MCMC(nuts_kernel, num_warmup=num_warmup, num_samples=num_samples, num_chains=num_chains, thinning=thinning, progress_bar=False)
    mcmc.warmup(prng_seed, X_train.P_i.values, X_train.D_ij.values, y_train, scaled_factors_origin, scaled_factors_destination, collect_warmup=True, extra_fields=("potential_energy",))
    poten = mcmc.get_extra_fields(group_by_chain=True)["potential_energy"]
    print("Chain 1", np.mean(-poten[0,:]))
    if num_chains==5:
        print("Chain 2", np.mean(-poten[1,:]))
        print("Chain 3", np.mean(-poten[2,:]))
        print("Chain 4", np.mean(-poten[3,:]))
        print("Chain 5", np.mean(-poten[4,:]))
    extra_fields = mcmc.get_extra_fields()
    assert "diverging" not in extra_fields, "Number of divergences: {}".format(jnp.sum(extra_fields["diverging"]))

    print("sampling posteriors for model of path {}/{}".format(path + 1, path_count))
    mcmc.run(prng_seed, X_train.P_i.values, X_train.D_ij.values, y_train, scaled_factors_origin, scaled_factors_destination)

    posteriors_gg += [neutra.transform_sample(mcmc.get_samples(group_by_chain=True)['auto_shared_latent'])]

    for name, value in posteriors_gg[-1].items():
        rhat = split_gelman_rubin(value).max()
        print("{} parameter has max R hat {}".format(name, rhat))
        # assert rhat < 1.1, "R hat too large"

    X_test = df_test.query("path_ind == @path").loc[:,'P_i':].astype('float')
    y_test = df_test.query("path_ind == @path").M_ij.values

    tmp = np.log(1 + X_test)
    scaled_factors = 1 / (1 + np.exp(- (tmp - tmp.mean(axis=0)) / tmp.std(ddof=1, axis=0) ))

    predictive = Predictive(reparam_ggm, posteriors_gg[-1], return_sites=["flow"])
    samples_predictive = predictive(prng_seed, X_test.P_i.values, X_test.D_ij.values, None, scaled_factors[features_origin].values, scaled_factors[features_destination].values)

    pred = np.exp( samples_predictive["flow"].mean(axis=0) ) - 1
    
    mae_gg[path] = mae(y_test, pred )
    r_squared_gg[path] = r_squared(y_test, pred)
    cpc_gg[path] = cpc(y_test, pred)
    cpcd_gg[path] = cpcd(y_test, pred, df_test.query("path_ind == @path").D_ij)
    
    print(mae_gg[path])

print("MAE: {}, +/- {}".format(mae_gg.mean(), norm.ppf(.975) * mae_gg.std(ddof=1) / np.sqrt(path_count) ) )
print("R-squared: {}, +/- {}".format(r_squared_gg.mean(), norm.ppf(.975) * r_squared_gg.std(ddof=1) / np.sqrt(path_count) ) )
print("CPC: {}, +/- {}".format(cpc_gg.mean(), norm.ppf(.975) * cpc_gg.std(ddof=1) / np.sqrt(path_count) ) )
print("CPCD: {}, +/- {}".format(cpcd_gg.mean(), norm.ppf(.975) * cpcd_gg.std(ddof=1) / np.sqrt(path_count) ) )

100%|██████████| 25000/25000 [00:15<00:00, 1662.28it/s, init loss: 726828.0506, avg. loss [23751-25000]: 70780.9158]


Chain 1 -65294.192347590935
Chain 2 -65177.47619206775
Chain 3 -65204.446788175155
Chain 4 -65136.35520489906
Chain 5 -65442.55790829731


AssertionError: Number of divergences: 24

In [5]:
mae_gg[0]

2645.8250455281195

In [6]:
features

{'population': ['P_i', 'P_j'],
 'housing': ['H_i', 'H_j'],
 'income': ['I_i', 'I_j'],
 'affordability': ['AF_i', 'AF_j'],
 'area': ['A_i', 'A_j'],
 'density': ['rho_i', 'rho_j'],
 'climate risk': ['ADC_i', 'ADC_j']}

In [7]:
posteriors_gg[0]['w'].reshape((-1,7)).mean(axis=0)

Array([0.43474673, 0.68037996, 0.01160111, 0.41280846, 1.47442949,
       0.04364658, 0.25848849], dtype=float64)

In [8]:
posteriors_gg[0]['w'].reshape((-1,7)).std(axis=0)

Array([0.14861701, 0.16268577, 0.18217934, 0.16209478, 0.27205569,
       0.12189468, 0.20443296], dtype=float64)

In [9]:
with open('../results/posteriors_gg.pkl', 'wb') as f:
    pickle.dump(posteriors_gg, f, protocol=pickle.HIGHEST_PROTOCOL)

### Generalized Radiation
$$
M_{i,j} = M_{i} \frac { U_i U_j  } {  (U_i + S^U_{i,j}) (U_{i} + U_{j} + S^U_{i,j})  }
$$
$$
U_i = \sum_k w_k f_{k:i}
$$
$$
M_i = \kappa P_i
$$

In [ ]:
def grm(X, y, scaled_factors, local_region_sum, features_origin, features_destination):
    
    # w_sigma = numpyro.sample("w_sigma", dist.HalfNormal(10*jnp.ones(len(features_origin))))
    # w_mu = numpyro.sample("w_mu", dist.Normal(0, 10*jnp.ones(len(features_origin))))
    # coeff_sigma = numpyro.sample("coeff_sigma", dist.HalfNormal(10*jnp.ones(6)))
    # coeff_mu = numpyro.sample("coeff_mu", dist.Normal(0, 10*jnp.ones(6)))

    # w = numpyro.sample("w", dist.Normal(0, 10*jnp.ones(len(features_origin))))
    # coeff = numpyro.sample("coeff", dist.Normal(coeff_mu, coeff_sigma))

    w = numpyro.sample("w", dist.Normal(0, 10*jnp.ones(len(features_origin))))
    coeff = numpyro.sample("coeff", dist.Normal(0, 10*jnp.ones(6)))
    sigma = numpyro.sample("sigma", dist.HalfNormal(10))

    U_i = jnp.dot(w , scaled_factors[features_origin].values.T)
    U_j = jnp.dot(w , scaled_factors[features_destination].values.T)
    SU_ij = jnp.dot(w, local_region_sum.T)

    M_ij = coeff[0] + jnp.dot(coeff[1:], jnp.array([np.log( 1 + X.P_i.values), jnp.log(1 + U_i) , jnp.log(1 + U_j),jnp.log(1 + U_i + SU_ij) , jnp.log(1 + U_i + U_j + SU_ij)  ]) )

    numpyro.sample("flow", dist.Normal(M_ij, sigma), obs=y)

    λ = numpyro.sample("λ", dist.HalfNormal(10))
    w = numpyro.sample("w", dist.Laplace(0, λ * jnp.ones(scaled_factors_origin.shape[1])))
    
    U_i = jnp.dot(w , scaled_factors_origin.T)
    U_j = jnp.dot(w , scaled_factors_destination.T)

    Β = jnp.array([1,1,1,-1,-1]) * numpyro.sample("Β", dist.HalfNormal(10*jnp.ones(5))) 
    σ = numpyro.sample("σ", dist.HalfNormal(10))

    M_ij = Β[0] + jnp.dot(Β[1:], jnp.array([jnp.log(1+U_i), jnp.log(1+U_j), jnp.log(1 + d_ij )]))

    numpyro.sample("flow", dist.Normal(M_ij, σ), obs=y)

In [ ]:
cpc_gr , cpcd_gr, mae_gr , r_squared_gr = np.empty(path_count), np.empty(path_count), np.empty(path_count), np.empty(path_count)
posteriors_gr = []
for path in df.path_ind.unique()[:path_count]:

    X_train = df_train.query("path_ind == @path")
    y_train = np.log(1 + df_train.query("path_ind == @path").M_ij.values)

    tmp = np.log(1 + X_train.loc[:,'P_i':])
    scaled_factors = pd.concat([X_train[['State_i','State_j']], 
                                1 / (1 + np.exp(- (tmp - tmp.mean(axis=0))\
                                    / tmp.std(ddof=1, axis=0) ))],
                                axis=1)

    features = {'population': ['P_i','P_j'],
            'housing': ['H_i','H_j'],
            'income': ['I_i','I_j'],
            'affordability': ['AF_i','AF_j'],
            'area': ['A_i','A_j'],
            'density': ['rho_i','rho_j'],
            'climate risk': ['ADC_i','ADC_j']
    }

    features_origin = list(np.array(list(features.values()))[:,0])
    features_destination = list(np.array(list(features.values()))[:,1])

    local_region_sum = np.empty((len(X_train), len(features)))
    for i in range(len(X_train)):
        intervening_states = states[distance[X_train.State_i.iloc[i]] < distance[X_train.State_i.iloc[i]][X_train.State_j.iloc[i]]]\
                            .drop(X_train.State_i.iloc[i])
        if len(intervening_states):
            local_region_sum[i] = np.array([scaled_factors.query("Year == {} & State_i == '{}'"\
                                .format(scaled_factors.index[i], local_state))[features_origin].values[0]
                                    for local_state in intervening_states]
                            ).sum(axis=0)
        else:
            local_region_sum[i] = np.array([0.]*len(features))

    guide = AutoDiagonalNormal(grm)
    optimizer = numpyro.optim.Adam(step_size=1e-2)
    svi = SVI(grm, guide, optimizer, Trace_ELBO())
    svi_result = svi.run(random.PRNGKey(0), 10000, X_train, y_train, scaled_factors, local_region_sum, features_origin, features_destination)
    neutra = NeuTraReparam(guide, svi_result.params)

    reparam_grm = neutra.reparam(grm)
    nuts_kernel = NUTS(reparam_grm)
    
    samples_num = 6000
    chains_num = 2
    mcmc = MCMC(nuts_kernel, num_warmup=1000, num_samples=samples_num//chains_num, num_chains=chains_num, progress_bar=False)
    mcmc.run(random.PRNGKey(0), X_train, y_train, scaled_factors, local_region_sum, features_origin, features_destination)

    posteriors_gr += [neutra.transform_sample(mcmc.get_samples(group_by_chain=True)['auto_shared_latent'])]

    X_test = df_test.query("path_ind == @path")
    y_test = df_test.query("path_ind == @path").M_ij.values

    tmp = np.log(1 + X_test.loc[:,'P_i':])
    scaled_factors = pd.concat([X_test[['State_i','State_j']], 
                                1 / (1 + np.exp(- (tmp - tmp.mean(axis=0))\
                                    / tmp.std(ddof=1, axis=0) ))],
                                axis=1)

    local_region_sum = np.empty((len(X_test), len(features)))
    for i in range(len(X_test)):
        intervening_states = states[distance[X_test.State_i.iloc[i]] < distance[X_test.State_i.iloc[i]][X_test.State_j.iloc[i]]]\
                            .drop(X_test.State_i.iloc[i])
        if len(intervening_states):
            local_region_sum[i] = np.array([scaled_factors.query("Year == {} & State_i == '{}'"\
                                .format(scaled_factors.index[i], local_state))[features_origin].values[0]
                                    for local_state in intervening_states]
                            ).sum(axis=0)
        else:
            local_region_sum[i] = np.array([0.]*len(features))

    predictive = Predictive(reparam_grm, posteriors_gr[-1], return_sites=["flow"])
    samples_predictive = predictive(random.PRNGKey(0), X_test, None, scaled_factors, local_region_sum, features_origin, features_destination)

    pred = np.exp( samples_predictive["flow"].mean(axis=0) ) - 1

    cpc_gr[path] = cpc(y_test, pred) 
    cpcd_gr[path] = cpcd(y_test, pred, df_test.query("path_ind == @path").D_ij) 
    mae_gr[path] = mae(y_test, pred )
    r_squared_gr[path] = r_squared(y_test, pred)

print("MAE: {}, +/- {}".format(mae_gr.mean(), norm.ppf(.975) * mae_gr.std(ddof=1) / np.sqrt(path_count) ) )
print("R-squared: {}, +/- {}".format(r_squared_gr.mean(), norm.ppf(.975) * r_squared_gr.std(ddof=1) / np.sqrt(path_count) ) )
print("CPC: {}, +/- {}".format(cpc_gr.mean(), norm.ppf(.975) * cpc_gr.std(ddof=1) / np.sqrt(path_count) ) )
print("CPCD: {}, +/- {}".format(cpcd_gr.mean(), norm.ppf(.975) * cpcd_gr.std(ddof=1) / np.sqrt(path_count) ) )

running path 0


sample: 100%|██████████| 4000/4000 [14:18<00:00,  4.66it/s, 1023 steps of size 5.98e-03. acc. prob=0.93]


completing path 0 took 44.0 minutes
running path 1


sample: 100%|██████████| 4000/4000 [13:52<00:00,  4.80it/s, 1023 steps of size 6.25e-03. acc. prob=0.92]


completing path 1 took 43.0 minutes
running path 2


...

completing path 2 took 43.0 minutes
running path 3


sample: 100%|██████████| 4000/4000 [14:27<00:00,  4.61it/s, 895 steps of size 5.71e-03. acc. prob=0.93] 


completing path 3 took 43.0 minutes
running path 4


sample: 100%|██████████| 4000/4000 [14:14<00:00,  4.68it/s, 1023 steps of size 5.76e-03. acc. prob=0.94]


completing path 4 took 44.0 minutes
MAE: 2309.5895611738692, +/- 15.527420777382872
R-squared: 0.18134906306973736, +/- 0.014495629715520248
CPC: 0.5119379292225815, +/- 0.001165777378622975
CPCD: 0.7500932521232849, +/- 0.005184211700233267


In [ ]:
with open('results/posteriors_gr.pkl', 'wb') as f:
    pickle.dump(posteriors_gr, f)